# W266 Authorship Attribution -- Data Retrieval

## Import Libraries and Set up dependencies

In [ ]:
import numpy as np 
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

### Clone Repo

In [ ]:
!git clone https://github.com/c-w/Gutenberg.git

fatal: destination path 'Gutenberg' already exists and is not an empty directory.


### Install BDS-DB3

In [ ]:
#don't use (Windows installation of bsddb3)
#!pip install bsddb3‑6.2.9‑pp37‑pypy37_pp73‑win_amd64.whl
#!pip install .

In [ ]:
#linux installation of bsddb3
!sudo apt-get install libdb++-dev
!export BERKELEYDB_DIR=/usr
!pip install .
!sudo apt autoremove

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libdb++-dev is already the newest version (1:5.3.21~exp1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


### Install Dependencies

In [ ]:
cd Gutenberg/

/content/Gutenberg


In [ ]:
!pip install virtualenv
!virtualenv --no-site-packages virtualenv
!. virtualenv/bin/activate
!pip install -r requirements-dev.pip
!pip install .

usage: virtualenv [--version] [--with-traceback] [-v | -q] [--read-only-app-data] [--app-data APP_DATA] [--reset-app-data] [--upgrade-embed-wheels] [--discovery {builtin}] [-p py] [--try-first-with py_exe]
                  [--creator {builtin,cpython3-posix,venv}] [--seeder {app-data,pip}] [--no-seed] [--activators comma_sep_list] [--clear] [--no-vcs-ignore] [--system-site-packages] [--symlinks | --copies] [--no-download | --download]
                  [--extra-search-dir d [d ...]] [--pip version] [--setuptools version] [--wheel version] [--no-pip] [--no-setuptools] [--no-wheel] [--no-periodic-update] [--symlink-app-data] [--prompt prompt] [-h]
                  dest
virtualenv: error: unrecognized arguments: --no-site-packages
SystemExit: 2
/bin/bash: virtualenv/bin/activate: No such file or directory
Processing /content/Gutenberg
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use -

### Load Metadata/Cache book metadata

In [ ]:
# Get book text by using book id
from gutenberg.acquire import load_etext
# Remove "The Project Gutenberg eBook" header text
from gutenberg.cleanup import strip_headers

In [ ]:
from gutenberg.query import list_supported_metadatas
print(list_supported_metadatas()) # prints (u'author', u'formaturi', u'language', ...)

('author', 'formaturi', 'language', 'rights', 'subject', 'title')


In [ ]:
from gutenberg.acquire import get_metadata_cache
cache = get_metadata_cache()
cache.populate()

### Create dataset with text data/inputs and author labels/outputs

In [ ]:
from pandas.core.dtypes.common import is_datetime64tz_dtype
from gutenberg.query import get_etexts
from gutenberg.query import get_metadata

sample_txt_ids = [
                  1513, 1524, 1533, 1112, 1514, 1531, 1522, 1532, 1515, 1519, # William Shakespeare
                  1342, 158, 161, 105, 121, 141, 946, 42078, 1212, # Jane Austen
                  98, 1400, 730, 766, 1023, 580, 883, 700, 967, 1289, 807, 25852, # Charles Dickens
                  205, 71, 1022, 34392, 42553, 4232, 34990, 42500, 9846, 60951, # Henry David Thoreau
                  140, 38117, 63293, 63693, 7522, 5421, 5776, 65576, 3304, 5774, # Upton Sinclair
                  64317, 805, 6695, 4368, 60962, # F. Scott Fitzgerald
                  1661, 2852, 244, 2097, 834, 139, 108, 903, 537, 2347, # Arthur Conan Doyle
                  16, 26654, 20914, 1376, 18934, 4021, 3490, 9617, 342, 6965, # J.M Barrie
                  76, 74, 86, 1837, 3176, 245, 3177, 3189, 3174, 119, # Mark Twain
                  84, 18247, 15238, 56665, 6447, 66749, 66545, 64329, 63337, 64555, # Mary Shelley
                  1497, 1656, 1672, 1750, 1600, 1636, 1726, 1658, 1642, 1572, # Plato
                  2148, 2147, 932, 1064, 32037, 1063, 2151, 2149, 17192, 2150, # Edgar Allan Poe
                  55, 520, 54, 22566, 4357, 33361, 419, 41667, 26624, 52176, # Lyman Frank Baum
                  408, 15210, 62582, 15265, 66398, 17700, 15359, 31254, 62799, 5685, # W.E.B. Du Bois
                  204, 1695, 1717, 1719, 130, 67639, 8092, 470, 223, 65688, # Gilbert Keith Chesterton
                  45, 47, 51, 544, 3796, 5343, 61236, 1354, 5342, 316, # Lucy Maude Montgomery
                  34568, 35601, 34550, 34737, 34582, 35013, 35070, 35157, 35097, 35128, # Thomas Tapper
                  25344, 13707, 77, 2081, 976, 32242, 513, 9243, 1916, 512, # Nathaniel Hawthorne
                  219, 220, 5658, 974, 2021, 17731, 1142, 1202, 495, 2480, # Joseph Conrad
                  863, 61262, 58866, 1155, 61168, 67173, 67160, 65238, 66446 # Agatha Christie
                  ]

def text_extractor(txt_id):
  txt = strip_headers(load_etext(etextno=txt_id))
  #return nltk.word_tokenize(txt)
  return txt

def metadata_getter(feat_name, txt_id):
  return str(list(get_metadata(feat_name, txt_id)))[2:-2]

data = {"text": [],
        "author" :[],
        "etxt_id" : [],
        "title" : [], 
        "subject" : []
        }

for id in sample_txt_ids:
  # Keep only etexts with singular attributed author
  assert len(get_metadata("author", id)) == 1, f"Author issue! Check etextno: {id}"
  data['text'].append(text_extractor(id))
  data['author'].append(metadata_getter('author', id))
  data["etxt_id"].append(id)
  data['title'].append(metadata_getter('title', id))
  data['subject'].append(metadata_getter('subject', id))

df = pd.DataFrame(data)

In [ ]:
author_vals = {k:v for k, v in list(zip(np.arange(0, len(pd.factorize(df.author)[1])), pd.factorize(df.author)[1]), )}
df["author_label"] = pd.factorize(df.author)[0]
df.head(50)

,text,author,etxt_id,title,subject,author_label
0,\n\n\n\nTHE TRAGEDY OF ROMEO AND JULIET\n\n\n\...,"Shakespeare, William",1513,Romeo and Juliet,"Youth -- Drama', 'Juliet (Fictitious character...",0
1,"\n\n\n\nTHE TRAGEDY OF HAMLET, PRINCE OF DENMA...","Shakespeare, William",1524,"Hamlet, Prince of Denmark","Tragedies', 'Denmark -- Drama', 'Fathers -- De...",0
2,\ncover \n\n\n\nMACBETH\n\nby William Shakespe...,"Shakespeare, William",1533,Macbeth,"Regicides -- Drama', 'Tragedies', 'Macbeth, Ki...",0
3,\n\n\n\n\n\n\n\n\n\n\n\n\n*Project Gutenberg i...,"Shakespeare, William",1112,The Tragedy of Romeo and Juliet,"Juliet (Fictitious character) -- Drama', 'Yout...",0
4,\ncover \n\n\n\n\nA MIDSUMMER NIGHT’S DREAM\n\...,"Shakespeare, William",1514,A Midsummer Night's Dream,"Athens (Greece) -- Drama', 'Comedies', 'Fairy ...",0
5,"\ncover \n\n\n\nOTHELLO, THE MOOR OF VENICE\n\...","Shakespeare, William",1531,"Othello, the Moor of Venice","Tragedies', 'Venice (Italy) -- Drama', 'Muslim...",0
6,\n\n\n\nTHE TRAGEDY OF JULIUS CAESAR\n\n\nby W...,"Shakespeare, William",1522,Julius Caesar,"Tragedies', 'Caesar, Julius -- Assassination -...",0
7,\n\n\n\nTHE TRAGEDY OF KING LEAR\n\nby William...,"Shakespeare, William",1532,The Tragedy of King Lear,"Lear, King (Legendary character) -- Drama', 'T...",0
8,\ncover\n\n\n\n\nTHE MERCHANT OF VENICE\n\n\nb...,"Shakespeare, William",1515,The Merchant of Venice,"Comedies', 'Moneylenders -- Drama', 'Venice (I...",0
9,\n\n\n\nMUCH ADO ABOUT NOTHING\n\nby William S...,"Shakespeare, William",1519,Much Ado about Nothing,"Messina (Italy) -- Drama', 'Comedies', 'Reject...",0


### Export dataset to designated Drive folder.

Set the working directory for this notebook for ease of access to files.

The command lines below assumes that the `W266 Term Project` folder is a shortcut in your Google Drive main/homepage (My Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pwd

/content/Gutenberg


In [ ]:
%cd ../gdrive/MyDrive/'W266 Term Project'

/content/gdrive/.shortcut-targets-by-id/1_FXrZ5ThkxncTxzIULW-MqLHa5Qo-qn6/W266 Term Project


In [119]:
# Export generated data to the designated folder
data = df.to_csv('data.csv')

In [120]:
author_vals

{0: 'Shakespeare, William',
 1: 'Austen, Jane',
 2: 'Dickens, Charles',
 3: 'Thoreau, Henry David',
 4: 'Sinclair, Upton',
 5: 'Fitzgerald, F. Scott (Francis Scott)',
 6: 'Doyle, Arthur Conan',
 7: 'Barrie, J. M. (James Matthew)',
 8: 'Twain, Mark',
 9: 'Shelley, Mary Wollstonecraft',
 10: 'Plato',
 11: 'Poe, Edgar Allan',
 12: 'Baum, L. Frank (Lyman Frank)',
 13: 'Du Bois, W. E. B. (William Edward Burghardt)',
 14: 'Chesterton, G. K. (Gilbert Keith)',
 15: 'Montgomery, L. M. (Lucy Maud)',
 16: 'Tapper, Thomas',
 17: 'Hawthorne, Nathaniel',
 18: 'Conrad, Joseph',
 19: 'Christie, Agatha'}

In [121]:
# Export `author_vals` dictionary for loading into pipeline/model evaluation
# Src: https://pythonspot.com/save-a-dictionary-to-a-file/
import pickle

# create a binary pickle file 
f = open("id2auth.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(author_vals,f)

# close file
f.close()

### Load exported data as Pandas DataFrame
As a check to see that the code worked and we get text that is only the book text (no Project Gutenberg headers).

In [ ]:
import_exported_data = pd.read_csv('data.csv')
import_exported_data.head()

,Unnamed: 0,text,author,etxt_id,title,subject,author_label
0,0,\n\n\n\nTHE TRAGEDY OF ROMEO AND JULIET\n\n\n\...,"Shakespeare, William",1513,Romeo and Juliet,"Youth -- Drama', 'Juliet (Fictitious character...",0
1,1,"\n\n\n\nTHE TRAGEDY OF HAMLET, PRINCE OF DENMA...","Shakespeare, William",1524,"Hamlet, Prince of Denmark","Tragedies', 'Denmark -- Drama', 'Fathers -- De...",0
2,2,\ncover \n\n\n\nMACBETH\n\nby William Shakespe...,"Shakespeare, William",1533,Macbeth,"Regicides -- Drama', 'Tragedies', 'Macbeth, Ki...",0
3,3,\n\n\n\n\n\n\n\n\n\n\n\n\n*Project Gutenberg i...,"Shakespeare, William",1112,The Tragedy of Romeo and Juliet,"Juliet (Fictitious character) -- Drama', 'Yout...",0
4,4,\ncover \n\n\n\n\nA MIDSUMMER NIGHT’S DREAM\n\...,"Shakespeare, William",1514,A Midsummer Night's Dream,"Athens (Greece) -- Drama', 'Comedies', 'Fairy ...",0


In [ ]:
import_exported_data[['text', 'author_label']]

,text,author_label
0,\n\n\n\nTHE TRAGEDY OF ROMEO AND JULIET\n\n\n\...,0
1,"\n\n\n\nTHE TRAGEDY OF HAMLET, PRINCE OF DENMA...",0
2,\ncover \n\n\n\nMACBETH\n\nby William Shakespe...,0
3,\n\n\n\n\n\n\n\n\n\n\n\n\n*Project Gutenberg i...,0
4,\ncover \n\n\n\n\nA MIDSUMMER NIGHT’S DREAM\n\...,0
...,...,...
190,\n\n\n\n THE MAN IN THE BROWN SUIT\n\n\n\n\n ...,19
191,\n\n The Missing Wil...,19
192,\n\n The Hunter’s Lodge ...,19
193,\n AGATHA\n CHRISTIE\n\n\n\n\n The Se...,19


Load our `author_vals` dictionary.

In [125]:
# Check to see our original author_vals dictionary is the same as our pickled/loaded dict
author_vals2 = pickle.load(open("id2auth.pkl", 'rb'))
author_vals == author_vals2

True

#### Notes/Misc Data Consistency Checks

ids 57837, 59813, 10947, 15041 don't have authors!

After investingating each id, these books are not actually by a singular author but instead are anthologies made up of many authors. So no authors are listed in the Gutenberg metadata.:

```
	Unnamed: 0	text	                    author	book_ids
35	35	      book was produced from...	4	      57837
37	37	      \n\n\n\nThe Footpath W...	4	      59813
83	83	      \n\n\n\nTHE BEST AMERI...	4		  10947
110	110	    \n\n\n\nTHE BEST AMERI...	4		  10947
128	128	    \n\n\n\n\n\n[Illustrat...	4		  15041
```

I've removed these exact ids and replaced them with non anthologies. I've also added a line of code to skip texts where the 'author' in `list_supported_metadatas()`/('author', 'formaturi', 'language', 'rights', 'subject', 'title') must only have one author (eliminating possibilities of multiple authors (in the case of co-authors) or no authors (in the case of anthology books).

In [ ]:
# Check for duplicated books
# Some books have multiple etext ids due to different book editions/versions so the titles are same but the id is different
import_exported_data[import_exported_data['title'].duplicated()]

,Unnamed: 0,text,author,etxt_id,title,subject,author_label


In [ ]:
# Check for duplicated ids
import_exported_data[import_exported_data['etxt_id'].duplicated()]

,Unnamed: 0,text,author,etxt_id,title,subject,author_label


In [ ]:
# Check that every row is unique
assert len(import_exported_data) == len(set(import_exported_data['etxt_id']))
# Check that every title is unique (we've removed all books with multple editions)
assert len(set(import_exported_data['etxt_id'])) == len(set(import_exported_data['title']))

In [ ]:
# Check that there are no extra or missing authors
assert len(set(import_exported_data['author'])) == len(author_vals)

True